In [ ]:

# script for producing self-produced streamlines/image streaks


import os
import cv2
import numpy as np
from PIL import Image


def load_image(image_path):
    """
    Load an image and convert it to a numpy array.
    """
    image = Image.open(image_path).convert('L')
    image_array = np.array(image).astype(np.float64)
    return image_array

def enhance_contrast(image, gamma=1.5):
    """
    Enhance the contrast of an image using histogram equalization followed by gamma correction.
    """
    # Histogram equalization
    image_uint8 = image.astype(np.uint8)
    equalized_image = cv2.equalizeHist(image_uint8)
    
    # Gamma correction
    gamma_corrected = np.power(equalized_image / 255.0, gamma) * 255.0

    gamma_corrected = np.clip(gamma_corrected, 0, 255).astype(np.uint8)
    
    return gamma_corrected


def add_text_to_frame(frame, text, position=(50, 55), font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=2, color=(0, 255, 255), thickness=3):
    """
    Add text label to the given frame using OpenCV.
    """
    if len(frame.shape) == 2:
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    frame = cv2.putText(frame, text, position, font, font_scale, color, thickness, cv2.LINE_AA)
    return frame

def create_running_average_animation(image_folder, start_idx, end_idx, window_size,
                                     output_video_file, background_image_path, background_alpha,
                                     contour_image_path, contour_alpha=0.5): # Added contour params
    """
    Create a running average animation with background and contour overlay using OpenCV.
    """
    # --- Load Background Image ---
    background_image_gray = load_image(background_image_path)
    # Ensure background image is in color
    background_image = cv2.cvtColor(background_image_gray.astype(np.uint8), cv2.COLOR_GRAY2BGR)

    # --- Load Contour Image ---
    contour_img_rgba = cv2.imread(contour_image_path, cv2.IMREAD_UNCHANGED)
    if contour_img_rgba is None:
        raise ValueError(f"Could not load contour image: {contour_image_path}")
    if len(contour_img_rgba.shape) != 3 or contour_img_rgba.shape[2] != 4:

        try:
            contour_pil = Image.open(contour_image_path).convert("RGBA")
            contour_img_rgba = cv2.cvtColor(np.array(contour_pil), cv2.COLOR_RGBA2BGRA)
            print(f"Contour image {contour_image_path} loaded via PIL and converted to BGRA.")
        except Exception as e:
             raise ValueError(f"Contour image {contour_image_path} must be loadable and have an alpha channel (4 channels). Error: {e}")

    contour_img_rgba = cv2.flip(contour_img_rgba, 0)

 
    image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.png')])[start_idx:end_idx + 1]
    num_images = len(image_files)

    if num_images < 1:
         raise ValueError(f"No images found in the specified range for folder: {image_folder}")
    if num_images < window_size:
        print(f'Warning: Not enough images ({num_images}) in the selected range for the full window size ({window_size}). Using available images.')
   

    images = []
    print(f"Loading and enhancing {num_images} images...")
    for image_path in image_files:
        img = load_image(image_path)
        img_contrast = enhance_contrast(img) # Enhance contrast for streaks
        images.append(img_contrast)
    print("Image loading complete.")

    height, width = images[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_file, fourcc, 30, (width, height), isColor=True)

    background_image = cv2.resize(background_image, (width, height))
    contour_img_rgba = cv2.resize(contour_img_rgba, (width, height))

    # --- Prepare Contour Alpha Mask ---
    contour_bgr = contour_img_rgba[:, :, :3]
    # Normalize contour's own alpha channel to 0-1
    contour_alpha_channel = contour_img_rgba[:, :, 3] / 255.0
    # Apply the desired overall transparency factor
    effective_contour_alpha = contour_alpha_channel * contour_alpha
    # Expand alpha mask to 3 channels for broadcasting during blending
    effective_contour_alpha_3ch = cv2.cvtColor((effective_contour_alpha * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR) / 255.0

    print("Starting video generation...")
    for i in range(num_images):
        current_start_idx = max(0, i - window_size + 1)
        window_images = images[current_start_idx:i + 1]

       
        window_images_thresholded = [np.where(img > 215, img, 0) for img in window_images]

        # Calculate average
        if not window_images_thresholded:
             continue
        avg_image = np.mean(window_images_thresholded, axis=0)

        
        min_val, max_val = np.min(avg_image), np.max(avg_image)
        if max_val > min_val:
             avg_image_norm = np.clip((avg_image - min_val) / (max_val - min_val) * 255, 0, 255).astype(np.uint8)
        else:
             avg_image_norm = np.zeros_like(avg_image, dtype=np.uint8)


        
        avg_image_color = cv2.cvtColor(avg_image_norm, cv2.COLOR_GRAY2BGR)

        blended_frame = cv2.addWeighted(avg_image_color, 1 - background_alpha, background_image, background_alpha, 0)


        blended_frame_float = blended_frame.astype(np.float32)
        contour_bgr_float = contour_bgr.astype(np.float32)

        # Perform alpha blending for contour overlay
        overlayed_frame_float = contour_bgr_float * effective_contour_alpha_3ch + blended_frame_float * (1.0 - effective_contour_alpha_3ch)

        # Convert final frame back to uint8
        final_frame = np.clip(overlayed_frame_float, 0, 255).astype(np.uint8)

 
        #  text display
        time_value = ((i + 3) / 5  - 1/5 )* (2 * 2 * np.pi * (8/60))
        text = f'{time_value:.2f} t f     [image {current_start_idx + 1}-{i + 1}]'
        frame_with_text = add_text_to_frame(final_frame, text)

        # Write frame multiple times for slower playback effect
        for _ in range(10):
            video_writer.write(frame_with_text)

        if (i + 1) % 50 == 0: # Print progress every 50 frames
             print(f"Processed frame {i+1}/{num_images}")


    video_writer.release()
    print(f'Animation saved to: {output_video_file}')


ridge_28 = 'C:/Users/Rafal/Documents/Master/piv_source/final/28.03/scaling/ridge/ridge.png'
canyon_28 = 'C:/Users/Rafal/Documents/Master/piv_source/final/28.03/scaling/canyon/canyon.png'
canyon_29 = 'C:/Users/Rafal/Documents/Master/piv_source/final/29.03/scaling/canyon/canyon_contour.png'
smooth_29 = 'C:/Users/Rafal/Documents/Master/piv_source/final/29.03/scaling/smooth/smooth_contour.png'

canyon_29_contour = 'C:/Users/Rafal/Documents/Master/piv_source/final/29.03/scaling/canyon/contours_only.png'
ridge_28_contour = 'C:/Users/Rafal/Documents/Master/piv_source/final/28.03/scaling/ridge/contours_only.png'
smooth_29_contour = 'C:/Users/Rafal/Documents/Master/piv_source/final/29.03/scaling/smooth/contours_only.png'
canyon_28_contour = 'C:/Users/Rafal/Documents/Master/piv_source/final/28.03/scaling/canyon/contours_only.png'

window_size = 5

In [14]:
### CANYON

In [5]:
background_alpha = 0.0
image_folder = 'E:/29.03/png/canyon_2f/run5'
motionType = '29_canyon_prograde_2f'

runNr = 3
start_idx = 18
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, canyon_29, background_alpha, canyon_29_contour, 0.19)


image_folder = 'E:/29.03/png/canyon_2f/run6'
motionType = '29_canyon_retrograde_2f'
runNr = 4
start_idx = 36
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, canyon_29, background_alpha, canyon_29_contour, 0.19)

Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501
Processed frame 100/501
Processed frame 150/501
Processed frame 200/501
Processed frame 250/501
Processed frame 300/501
Processed frame 350/501
Processed frame 400/501
Processed frame 450/501
Processed frame 500/501
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_29_canyon_prograde_2f_3_5frames_contour.mp4
Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501
Processed frame 100/501
Processed frame 150/501
Processed frame 200/501
Processed frame 250/501
Processed frame 300/501
Processed frame 350/501
Processed frame 400/501
Processed frame 450/501
Processed frame 500/501
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_29_canyon_retrograde_2f_4_5frames_contour.mp4


In [17]:
background_alpha = 0.0
image_folder = 'E:/28.03/png/canyon_1f/run3'
motionType = '28_canyon_prograde_1f'
runNr = 3
start_idx = 41
end_idx = 500 + start_idx #int(input("Enter the end index of the images to use: "))
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, canyon_28, background_alpha, canyon_28_contour, 0.19)


image_folder = 'E:/28.03/png/canyon_1f/run6'
motionType = '28_canyon_retrograde_1f'
runNr = 6
start_idx = 27
end_idx = 500 + start_idx #int(input("Enter the end index of the images to use: "))
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, canyon_28, background_alpha, canyon_28_contour, 0.19)

Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501
Processed frame 100/501
Processed frame 150/501
Processed frame 200/501
Processed frame 250/501
Processed frame 300/501
Processed frame 350/501
Processed frame 400/501
Processed frame 450/501
Processed frame 500/501
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_28_canyon_prograde_1f_3_5frames_contour.mp4
Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501
Processed frame 100/501
Processed frame 150/501
Processed frame 200/501
Processed frame 250/501
Processed frame 300/501
Processed frame 350/501
Processed frame 400/501
Processed frame 450/501
Processed frame 500/501
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_28_canyon_retrograde_1f_6_5frames_contour.mp4


In [18]:
### RIDGE

In [3]:
background_alpha = 0.0
image_folder = 'E:/28.03/png/ridge_1f/run1'
motionType = '28_ridge_prograde_1f'
runNr = 3
start_idx = 24
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, ridge_28, background_alpha, ridge_28_contour, 0.19)


image_folder = 'E:/28.03/png/ridge_1f/run2'
motionType = '28_ridge_retrograde_1f'
runNr = 4
start_idx = 31
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, ridge_28, background_alpha, ridge_28_contour, 0.19)


image_folder = 'E:/28.03/png/ridge_2f/run3'
motionType = '28_ridge_prograde_2f'
runNr = 1
start_idx = 33
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, ridge_28, background_alpha, ridge_28_contour, 0.19)


image_folder = 'E:/28.03/png/ridge_2f/run2'
motionType = '28_ridge_retrograde_2f'
runNr = 4
start_idx = 29
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, ridge_28, background_alpha, ridge_28_contour, 0.19)

Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501
Processed frame 100/501
Processed frame 150/501
Processed frame 200/501
Processed frame 250/501
Processed frame 300/501
Processed frame 350/501
Processed frame 400/501
Processed frame 450/501
Processed frame 500/501
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_28_ridge_prograde_1f_3_5frames_contour.mp4
Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501
Processed frame 100/501
Processed frame 150/501
Processed frame 200/501
Processed frame 250/501
Processed frame 300/501
Processed frame 350/501
Processed frame 400/501
Processed frame 450/501
Processed frame 500/501
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_28_ridge_retrograde_1f_4_5frames_contour.mp4
Loading and enhancing 501 images...
Image loading complete.
Starting video generation.

In [20]:
### SMOOTH

In [4]:
background_alpha = 0.0
image_folder = 'E:/29.03/png/smooth_1f/run1'
motionType = '29_smooth_prograde_1f'
runNr = 3
start_idx = 20
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, smooth_29, background_alpha, smooth_29_contour, 0.19)


image_folder = 'E:/29.03/png/smooth_1f/run2'
motionType = '29_smooth_retrograde_1f'
runNr = 4
start_idx = 23
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, smooth_29, background_alpha, smooth_29_contour, 0.19)


image_folder = 'E:/29.03/png/smooth_2f/run1'
motionType = '29_smooth_prograde_2f'
runNr = 3
start_idx = 28
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, smooth_29, background_alpha, smooth_29_contour, 0.19)


image_folder = 'E:/29.03/png/smooth_2f/run2'
motionType = '29_smooth_retrograde_2f'
runNr = 4
start_idx = 28
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour.mp4' 
create_running_average_animation(image_folder, start_idx, end_idx, window_size, output_video_file, smooth_29, background_alpha, smooth_29_contour, 0.19)

Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501
Processed frame 100/501
Processed frame 150/501
Processed frame 200/501
Processed frame 250/501
Processed frame 300/501
Processed frame 350/501
Processed frame 400/501
Processed frame 450/501
Processed frame 500/501
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_29_smooth_prograde_1f_3_5frames_contour.mp4
Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501
Processed frame 100/501
Processed frame 150/501
Processed frame 200/501
Processed frame 250/501
Processed frame 300/501
Processed frame 350/501
Processed frame 400/501
Processed frame 450/501
Processed frame 500/501
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_29_smooth_retrograde_1f_4_5frames_contour.mp4
Loading and enhancing 501 images...
Image loading complete.
Starting video generatio

In [ ]:

def add_text_to_frame(frame, text, position=(50, 90), font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=2, color=(0, 255, 255), thickness=3):
    """
    Add text label to the given frame using OpenCV.
    """
    if len(frame.shape) == 2:
        
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    frame = cv2.putText(frame, text, position, font, font_scale, color, thickness, cv2.LINE_AA)
    return frame

def create_running_average_animation(image_folder, start_idx, end_idx, window_size,
                                     output_video_file, 
                                     contour_image_path=None, contour_alpha=0.5):
    """
    Create a running average animation of images.
    Optionally overlays a contour image.
    """
    
    image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')])[start_idx:end_idx + 1]
    num_images = len(image_files)

    if num_images < 1:
         raise ValueError(f"No images found in the specified range for folder: {image_folder}")
    if num_images < window_size:
        print(f'Warning: Not enough images ({num_images}) in the selected range for the full window size ({window_size}). Using available images.')

    images = []
    print(f"Loading and enhancing {num_images} images...")
    for image_path_iter in image_files:
        img = load_image(image_path_iter)
        img_contrast = enhance_contrast(img) 
        images.append(img_contrast)
    print("Image loading complete.")

    height, width = images[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_file, fourcc, 30, (width, height), isColor=True)

    
    contour_bgr = None
    effective_contour_alpha_3ch = None

    if contour_image_path:
        contour_img_rgba = cv2.imread(contour_image_path, cv2.IMREAD_UNCHANGED)
        if contour_img_rgba is None:
            print(f"Warning: Could not load contour image: {contour_image_path}. Proceeding without contours.")
        elif len(contour_img_rgba.shape) != 3 or contour_img_rgba.shape[2] != 4:
            try:
                contour_pil = Image.open(contour_image_path).convert("RGBA")
                contour_img_rgba = cv2.cvtColor(np.array(contour_pil), cv2.COLOR_RGBA2BGRA)
                print(f"Contour image {contour_image_path} loaded via PIL and converted to BGRA.")
            except Exception as e:
                 print(f"Warning: Contour image {contour_image_path} must be loadable and have an alpha channel. Error: {e}. Proceeding without contours.")
                 contour_img_rgba = None 
        
        if contour_img_rgba is not None:
            contour_img_rgba = cv2.flip(contour_img_rgba, 0) 
            contour_img_rgba = cv2.resize(contour_img_rgba, (width, height))
            contour_bgr = contour_img_rgba[:, :, :3]
            contour_alpha_channel = contour_img_rgba[:, :, 3] / 255.0
            effective_contour_alpha_val = contour_alpha_channel * contour_alpha 
            effective_contour_alpha_3ch = cv2.cvtColor((effective_contour_alpha_val * 255).astype(np.uint8), cv2.COLOR_GRAY2BGR) / 255.0
    
    print("Starting video generation...")
    for i in range(num_images):
        current_start_idx = max(0, i - window_size + 1)
        window_images_slice = images[current_start_idx:i + 1]

       
        shadow_threshold = 225 
        window_images_thresholded = [np.where(img > shadow_threshold, img, 0) for img in window_images_slice]

        if not window_images_thresholded:
             continue
        avg_image = np.mean(window_images_thresholded, axis=0)

        min_val, max_val = np.min(avg_image), np.max(avg_image)
        if max_val > min_val:
             avg_image_norm = np.clip((avg_image - min_val) / (max_val - min_val) * 255, 0, 255).astype(np.uint8)
        else:
             avg_image_norm = np.zeros_like(avg_image, dtype=np.uint8)

        avg_image_color = cv2.cvtColor(avg_image_norm, cv2.COLOR_GRAY2BGR)
        
        final_frame = avg_image_color 

        
        if contour_bgr is not None and effective_contour_alpha_3ch is not None:
            base_frame_float = final_frame.astype(np.float32)
            contour_bgr_float = contour_bgr.astype(np.float32)
            overlayed_frame_float = contour_bgr_float * effective_contour_alpha_3ch + base_frame_float * (1.0 - effective_contour_alpha_3ch)
            final_frame = np.clip(overlayed_frame_float, 0, 255).astype(np.uint8)

        # Add text
        time_value = ((i + 3) / 5  - 1/5 )* (2 * 2 * np.pi * (8/60)) 
        text = f'[image {current_start_idx +1}-{i + 1}]' 
        frame_with_text = add_text_to_frame(final_frame, text)

        for _ in range(10):
            video_writer.write(frame_with_text)

        if (i + 1) % 50 == 0:
             print(f"Processed frame {i+1}/{num_images} (Input image index {i + start_idx +1})")

    video_writer.release()
    print(f'Animation saved to: {output_video_file}')


In [ ]:
image_folder = 'C:/Users/Rafal/Documents/Master/piv_source/10.12/run3'
motionType = 'ikea_prograde_1rpm'
runNr = 3 
start_idx = 100
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour_noBG.mp4' # Added _noBG

output_video_file_no_contour = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_noBG_noContour.mp4'

create_running_average_animation(
    image_folder, 
    start_idx, 
    end_idx, 
    window_size, 
    output_video_file_no_contour
)

Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501 (Input image index 150)
Processed frame 100/501 (Input image index 200)
Processed frame 150/501 (Input image index 250)
Processed frame 200/501 (Input image index 300)
Processed frame 250/501 (Input image index 350)
Processed frame 300/501 (Input image index 400)
Processed frame 350/501 (Input image index 450)
Processed frame 400/501 (Input image index 500)
Processed frame 450/501 (Input image index 550)
Processed frame 500/501 (Input image index 600)
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_ikea_prograde_1rpm_3_5frames_noBG_noContour.mp4


In [24]:
image_folder = 'C:/Users/Rafal/Documents/Master/piv_source/10.12/run4'
motionType = 'ikea_retrograde_1rpm'
runNr = 4 # Example, ensure this is correct for your naming
start_idx = 66
end_idx = 500 + start_idx 
output_video_file = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_contour_noBG.mp4' # Added _noBG

output_video_file_no_contour = f'C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_{motionType}_{runNr}_5frames_noBG_noContour.mp4'

create_running_average_animation(
    image_folder, 
    start_idx, 
    end_idx, 
    window_size, 
    output_video_file_no_contour
)

Loading and enhancing 501 images...
Image loading complete.
Starting video generation...
Processed frame 50/501 (Input image index 116)
Processed frame 100/501 (Input image index 166)
Processed frame 150/501 (Input image index 216)
Processed frame 200/501 (Input image index 266)
Processed frame 250/501 (Input image index 316)
Processed frame 300/501 (Input image index 366)
Processed frame 350/501 (Input image index 416)
Processed frame 400/501 (Input image index 466)
Processed frame 450/501 (Input image index 516)
Processed frame 500/501 (Input image index 566)
Animation saved to: C:/Users/Rafal/Documents/Master/main/results/final/image_streaks_ikea_retrograde_1rpm_4_5frames_noBG_noContour.mp4
